In [1]:
import ee
ee.Initialize(project='pyregence-ee')
import datetime
import scripts.analysis_functions as af
import scripts.utils as utils
%load_ext autoreload
%autoreload 2

### Run BS Mapper for a particular year

In [ ]:
#
#
# TODO

##### IMPORT DATA #####
# One set of yearly fires, no need for remove_recent() or year/ac filtering, that was already done upon data fetch

#***
# USER: Pick either NICF or MTBS import variables, select year that matches with data import (manual)
#***

yr = '2020'

# # NIFC option
# fires_asset = f"projects/pyregence-ee/assets/conus/nifc/nifc_fires_{yr}_gte100ac_20230406"
# fires = ee.FeatureCollection(fires_asset)
# desc = f'nifc_bs_{yr}_gte100acres_20230406'
# asset_folder = 'projects/pyregence-ee/assets/conus/nifc'

# #MTBS option
# fires = ee.FeatureCollection("projects/pyregence-ee/assets/conus/mtbs/mtbs_fires_2020_20230406")
# desc = f'mtbs_bs_2020_20230406'
# asset_folder = 'projects/pyregence-ee/assets/conus/mtbs'

# Test / development case, limited numbers of fires
fires_raw = ee.FeatureCollection('projects/pyregence-ee/assets/conus/mtbs/mtbs_fires_2021_20230810') #get correct year 2020? TODO
irwin_list = ['ACD5C207-BB83-4419-977D-3F64778081A3', '499D6545-2A19-458A-8C7C-554932C6B86E'] # get 2020 irwin IDs TODO
    #BOBCAT = , DOME = , CASTLE = , 
fires = fires_raw.filter(ee.Filter.inList('irwinID',irwin_list))
desc = f'test_20230xxx'
asset_folder = 'projects/pyregence-ee/assets/fires_bs_tool/dmn_testing'

print(f'Total Fires in FeatureCollection: {fires.size().getInfo()}')


##### SET (SIMULATED) RUN DATE #####
#  TODO

# simulation_date = 

# def set_sim_date(ee.Feature)

# cut off date filtering needed?? TODO
#     fires = fires.filter(ee.Filter.lte('Discovery','2022-07-01'))
#     print(fires.sort('Discovery',False).first().getInfo()['properties'])
#     print(f'Fires after filter: {fires.size().getInfo()}')



##### RUN BURN SEVERITY #####

# return BS ee.Image for each fire ee.Feature
bs_coll = ee.FeatureCollection(fires).map(af.bs_calc_v2309)

# Export intermediate: individual features, severity classified
# TODO

# composite that ee.ImageCollection with a max() reducer
bs_composite = ee.ImageCollection(bs_coll).max().rename('SEVERITY').set('Year',int(yr)) 


##### EXPORT FINAL #####

# To Asset

utils.exportImgtoAsset(bs_composite, 
                    desc=desc,
                    region=None,
                    asset_folder= asset_folder, 
                    export_type='conus',
                    export=True)
                    #default=sensor) # TODO check if need to add sensor here, for NICF vs MTBS
       

# For one fire 
### Select a fire from a pre-existing fire featurecollection or provide your own fire feature asset (must have 'Discovery' property with value of format ee.String('yyyy-mm-dd') )


In [2]:
# Running for a particular fire

# Select the fire based on irwinID (or name and edit filter below)
#fire_name = 'KNP COMPLEX' 
#fire_irwin = '499D6545-2A19-458A-8C7C-554932C6B86E'
#fire_name = 'WINDY' #there are 2 WINDY fires, using irwinID # fire_irwin = 'ACD5C207-BB83-4419-977D-3F64778081A3'
fire_irwin = 'ACD5C207-BB83-4419-977D-3F64778081A3'

# Read in fire data
fires = ee.FeatureCollection('projects/pyregence-ee/assets/conus/mtbs/mtbs_fires_2021_20230810')
print(f'Total Fires in FeatureCollection: {fires.size().getInfo()}')
#print(fires.first().getInfo()['properties'])
#print(fires.aggregate_array('Incid_Name').distinct().sort().getInfo())

# Filter to desired fire
#fires_f = fires.filter(ee.Filter.eq('Incid_Name',fire_name))
fires_f = fires.filter(ee.Filter.eq('irwinID',fire_irwin))
print(f'Filtered fire size: {fires_f.size().getInfo()}')

fire_name = fires_f.first().get('Incid_Name').getInfo().lower() #brittle, make sure only 1 feature
print(fire_name)

# Run burn severity tool 
bs_coll = ee.FeatureCollection(fires_f).map(af.bs_calc_new)
#bs_fire = af.bs_calc_new(fires_f.first()) #testing direct function call
#print(bs_fire.propertyNames().getInfo())
# while max() isn't really needed for 1 fire, keeping the pattern of the multi-fire runs
bs_composite = ee.ImageCollection(bs_coll).max().add(1).rename('SEVERITY') #shift severity values +1 so 0 can be nodata

# To Asset
# get current date to put into asset name
cur_date = datetime.datetime.now().strftime("%Y%m%d")
desc = f'mtbs_bs_{str(fire_name).replace(" ", "")}_{str(cur_date)}'
print(desc)
#fails with Image.clipToBoundsAndScale: The geometry for image clipping must be bounded.
utils.exportImgtoAsset(bs_composite,  
                    desc=desc,
                    region=bs_coll.geometry(),
                    asset_folder='projects/pyregence-ee/assets/fires_bs_tool/dmn_testing', 
                    export_type='single_fire', 
                    export=False) 


Total Fires in FeatureCollection: 1022
Filtered fire size: 1
windy


In [8]:
# Running for a particular fire - ONLY CALC RDNBR

# Select the fire based on irwinID 
    #fire_name = 'KNP COMPLEX' 
#fire_irwin = '499D6545-2A19-458A-8C7C-554932C6B86E'
    #fire_name = 'WINDY' #there are 2 WINDY fires, using irwinID # fire_irwin = 'ACD5C207-BB83-4419-977D-3F64778081A3'
fire_irwin = 'ACD5C207-BB83-4419-977D-3F64778081A3'

# Read in fire data
fires = ee.FeatureCollection('projects/pyregence-ee/assets/conus/mtbs/mtbs_fires_2021_20230810')
print(f'Total Fires in FeatureCollection: {fires.size().getInfo()}')
#print(fires.first().getInfo()['properties'])
#print(fires.aggregate_array('Incid_Name').distinct().sort().getInfo())

# Filter to desired fire
#fires_f = fires.filter(ee.Filter.eq('Incid_Name',fire_name))
fires_f = fires.filter(ee.Filter.eq('irwinID',fire_irwin))
print(f'Filtered fire size: {fires_f.size().getInfo()}')

fire_name = fires_f.first().get('Incid_Name').getInfo().lower() #brittle, make sure only 1 feature
print(fire_name)

# CalC RDNBR
rdnbr_coll = ee.FeatureCollection(fires_f).map(af.rdnbr_only_calc)
#bs_fire = af.bs_calc_new(fires_f.first()) #testing direct function call
#print(bs_fire.propertyNames().getInfo())
# while max() isn't really needed for 1 fire, keeping the pattern of the multi-fire runs
rdnbr_composite = ee.ImageCollection(rdnbr_coll).max().rename('RDNBR') 

# To Asset
# get current date to put into asset name
cur_date = datetime.datetime.now().strftime("%Y%m%d")
desc = f'mtbs_bs_rdnbr_{str(fire_name).replace(" ", "")}_{str(cur_date)}'
print(desc)
#fails with Image.clipToBoundsAndScale: The geometry for image clipping must be bounded.
utils.exportImgtoAsset(rdnbr_composite,  
                    desc=desc,
                    region=rdnbr_coll.geometry(),
                    asset_folder='projects/pyregence-ee/assets/fires_bs_tool/dmn_testing', 
                    export_type='single_fire', 
                    export=False)


Total Fires in FeatureCollection: 1022
Filtered fire size: 1
knp complex
mtbs_bs_rdnbr_knpcomplex_20230829
export task started: projects/pyregence-ee/assets/fires_bs_tool/dmn_testing/mtbs_bs_rdnbr_knpcomplex_20230829


# ee authentication block

In [ ]:
# import ee
# ee.Authenticate()